
# 1. OpenAI VLM (GPT) - Basics
This section demonstrates the basic usage of OpenAI's Vision Language Model (VLM) capabilities using GPT-4.1.
We will use the OpenAI API to analyze an image and provide detailed textual insights.

**Support Material**

- https://platform.openai.com/docs/quickstart
- https://platform.openai.com/docs/guides/text
- https://platform.openai.com/docs/guides/images-vision?api-mode=chat
- https://platform.openai.com/docs/guides/structured-outputs


In [2]:
import openai
from dotenv import load_dotenv  
import os
import base64
import json
import textwrap

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


load_dotenv()
openAIclient = openai.OpenAI()


# Path to your image
img = "images/street_scene.jpg"




In [16]:
#basic call to gpt with prompt and image

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))


The image depicts a vibrant urban street scene in a city with tall buildings. There are various people engaged in
different activities:   - One person is sitting on the sidewalk using a tablet or smartphone. - Another person is lying
down on the pavement nearby. - A man dressed in a suit and a young woman are sitting on a wooden bench, both appearing
thoughtful or engaged in reading. - Several pigeons are on the pavement and near the bench. - In the background, a
motorbike rider and a person playing a guitar are crossing the street, along with cars driving by. - A woman is walking
on the sidewalk, engrossed in her phone. - The cityscape in the background has modern skyscrapers and older style
buildings, suggesting a blend of old and new architecture. - A traffic light shows a red signal for vehicles.  The
overall scene is lively, with a mix of pedestrians, vehicles, and street elements contributing to the daily rhythm of
city life.



# 1.1 Structured Output
Here, we expand upon the VLM example to request structured outputs. This approach allows for extracting 
well-organized information from images in a machine-readable format, such as JSON.

**Support Material**:
- https://platform.openai.com/docs/guides/structured-outputs


In [18]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={ "type": "json_object" },# NEW!!
    temperature = 0
)

returnValue = completion.choices[0].message.content


KeyboardInterrupt: 

We parse the json in a dict structure:

In [20]:
output = json.loads(returnValue)

So we can access specific infos:

In [21]:
output["description"]["foreground"]

KeyError: 'description'


# JSON Schema for Controlled Structured Outputs
In this section, we define a JSON schema for a more controlled and specific output from the model. 
Using this schema, we can ensure the model adheres to predefined data types and structures while describing images.In this case we will provide the json schema directly.



In [3]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={
                "type": "json_schema",    
                "json_schema": {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc."
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc."
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]
                    }}},
    temperature = 0
)

returnValue = completion.choices[0].message.content


In [4]:
output_image_extraction = json.loads(returnValue)

In [12]:
output_image_extraction["people"]

[{'position': 'sitting on the ground near a flower pot',
  'age': 16,
  'activity': 'using a smartphone',
  'gender': 'male'},
 {'position': 'lying on the ground',
  'age': 18,
  'activity': 'resting or relaxing',
  'gender': 'male'},
 {'position': 'sitting on a bench',
  'age': 65,
  'activity': 'thinking or resting',
  'gender': 'male'},
 {'position': 'sitting on a bench',
  'age': 25,
  'activity': 'reading a newspaper',
  'gender': 'female'},
 {'position': 'walking on the sidewalk',
  'age': 20,
  'activity': 'using a smartphone',
  'gender': 'female'},
 {'position': 'riding a motorcycle',
  'age': 30,
  'activity': 'driving',
  'gender': 'male'},
 {'position': 'walking on the street',
  'age': 28,
  'activity': 'playing guitar',
  'gender': 'male'},
 {'position': 'riding a scooter',
  'age': 27,
  'activity': 'driving',
  'gender': 'female'},
 {'position': 'inside a taxi car',
  'age': 40,
  'activity': 'driving',
  'gender': 'male'},
 {'position': 'inside a taxi car',
  'age': 35

Alternatively: 


OpenAI SDKs for Python and JavaScript also make it easy to define object schemas using Pydantic and Zod respectively. Below, you can see how to extract information from unstructured text that conforms to a schema defined in code.


In [ ]:
from pydantic import BaseModel


class Person(BaseModel):
    position: str 
    age: int 
    activity: str 
    gender: str


class ImageExtraction(BaseModel):
    number_of_people: int 
    atmosphere: str 
    hour_of_the_day: int 
    people: list[Person] 

completion = openAIclient.chat.completions.(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": "describe the image in detail"}
    ],
    response_format=ImageExtraction,
)

output_image_extraction = completion.choices[0].message.parsed

We can then integrate the extracted information in full or partially in a new prompt for a new extraction

In [5]:
#alert service prompt 

alert_sys_prompt = " you are an experienced first aid paramedical"
alert_prompt= """Extract from the following scene analysis give to you in json format, 
if anyone might be in danger and if the Child Hospital or normal Hospital should be alerted. 
Give the a concise answer
The situation is given to you from this object: """ + str(output_image_extraction)


In [8]:

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "user", "content": alert_prompt},
        {"role": "user", "content": alert_prompt}
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))

No one appears to be in immediate danger based on the given information. No hospital, child or normal, needs to be
alerted.


In [14]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Considering this list of people"+str(output_image_extraction["people"])+".Identify the youngest in the list and give me back their coordinate in the image I provide"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))

The youngest person in the list is the 16-year-old male sitting on the ground near a flower pot using a smartphone. In
the image, he is located at the bottom left corner near the flower pot. The approximate coordinate for this person in
the image is (150, 770).



# 2. Google VLM (Gemini)
This section demonstrates the use of Google's Vision Language Model, Gemini. 
We explore basic text generation as well as its ability to analyze images and provide relevant outputs.

**Support Material**:
- https://ai.google.dev/gemini-api/docs/quickstart
- https://ai.google.dev/gemini-api/docs/text-generation
- https://ai.google.dev/gemini-api/docs/image-understanding
- https://ai.google.dev/gemini-api/docs/structured-output?example=recipe

In [15]:
%matplotlib inline
import os
from dotenv import load_dotenv  
from google import genai
from PIL import Image

load_dotenv()
client = genai.Client()

Basic call:

In [17]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works to a 90 years old. in few words"
)
print(response.text)

Imagine a very clever computer that learns from **lots** of examples, just like we learn from experience. It then uses what it's learned to help us, recognize things, or make good guesses.


and with images: 

In [12]:
im = Image.open(img)

response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Detect if there is a person who is under 18 years old on the floor and reutrn its coordinates as a list in the format '[ymin,xmin, ymax, xmax]'. Jut output the list.\n"],
                                          )


print(response.text)


[680, 321, 963, 592]


Also here we can extract structured output 

Let's try to use Gemini to detect an object in the image and get its coordinates:


Gemini2+ was trained specifically for object detection/ segmentation tasks. More details: https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Spatial_understanding.ipynb
